In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from keras.layers import Input,Lambda,Dense,Flatten,Conv2D
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
import random

In [ ]:
df = pd.read_csv(r"../input/cassava-leaf-disease-classification/train.csv")
df.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(y='label',data=df)
plt.show()

In [ ]:
plt.figure(figsize=(22,25))
path = '../input/cassava-leaf-disease-classification/train_images/'
temp = df[df['label']==1]['image_id']  #all the images of dogs will be stored in temp
start = random.randint(0,len(temp))  #this generates a random number between 0 and len(temp)
files = temp[start:start+25]         #in files we store the first 25 continuous images between the random number generated 
                                     #since temp is basically images of dogs from step 2
for index,file in enumerate(files):
  plt.subplot(5,5,index+1)
  img = load_img(path+file)
  img = np.array(img)
  plt.imshow(img)
  plt.imshow(img)
  plt.title('Casava Leaf')
  plt.axis()

In [ ]:
WORK_DIR = '../input/cassava-leaf-disease-classification'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
import cv2

Total number of images in train_images folder

In [ ]:
import os
print(len(os.listdir('../input/cassava-leaf-disease-classification/train_images')))

In [ ]:
df.label = df.label.astype('str')

train_datagen = ImageDataGenerator(validation_split = 0.2,
                                     preprocessing_function = None,
                                     rotation_range = 45,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1)

train_generator = train_datagen.flow_from_dataframe(df,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (224,224),
                         batch_size = 32,
                         class_mode = "categorical")


validation_datagen = ImageDataGenerator(validation_split = 0.2)

validation_generator = validation_datagen.flow_from_dataframe(df,
                         directory = os.path.join(WORK_DIR, "train_images"),
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (224, 224),
                         batch_size = 32,
                         class_mode = "categorical")

In [ ]:
vgg = VGG16(input_shape=[224,224,3],weights='imagenet',include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
vgg_output = Dense(5,activation='softmax')(x)
#This is the last layer cnsisting of the various classes we are replacing the include_top = False layer with this one

In [ ]:
len(df)

In [ ]:
model = Model(inputs=vgg.input, outputs=vgg_output)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(validation_generator)
)

In [ ]:
#loss
plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label = 'val loss')
plt.legend()
plt.show()

In [ ]:
#accuracy
plt.plot(history.history['accuracy'],label='train acc')
plt.plot(history.history['val_accuracy'],label='val acc')
plt.legend()
plt.show()

In [ ]:
ss = pd.read_csv(os.path.join(WORK_DIR, "sample_submission.csv"))
ss

In [ ]:
preds = []

for image_id in ss.image_id:
    image = Image.open(os.path.join(WORK_DIR,  "test_images", image_id))
    image = image.resize((224,224))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))

ss['label'] = preds
ss

In [ ]:
ss.to_csv('submission.csv', index = False)


In [ ]:
from keras.models import Model
model.save('cassava.h5')